<a href="https://colab.research.google.com/github/nattrio/NF-kB-prediction/blob/main/get_smiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# File
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
df = pd.read_csv("/content/drive/My Drive/[0]Study/Bio-Comp/Gene/GeneID_4790_bioactivity_gene.csv", header=0)
df.info()
# df.groupby(['activity']).count()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (14,21) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238850 entries, 0 to 238849
Data columns (total 24 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   baid        238850 non-null  int64  
 1   activity    238850 non-null  object 
 2   aid         238850 non-null  int64  
 3   sid         238850 non-null  int64  
 4   mid         238850 non-null  int64  
 5   cid         238850 non-null  int64  
 6   geneid      238850 non-null  int64  
 7   taxid       238850 non-null  int64  
 8   pmid        1036 non-null    float64
 9   aidtype     238850 non-null  object 
 10  aidmdate    238850 non-null  int64  
 11  hasdrc      238850 non-null  int64  
 12  rnai        238850 non-null  int64  
 13  protacxn    238850 non-null  object 
 14  acname      25579 non-null   object 
 15  acvalue     4679 non-null    float64
 16  aidsrcname  238850 non-null  object 
 17  aidname     238850 non-null  object 
 18  cmpdname    238850 non-null  object 
 19  ta

In [3]:
df = df.loc[df['activity'].isin(["Active", "Inactive"])]
# df.info()

In [4]:
def isActive(str):
    if str == "Active":
        return 1
    elif str == "Inactive": 
        return 0

In [5]:
df["class"] = df["activity"]
class_data = df['class'].to_numpy()

for i in range(len(class_data)):
    class_data[i] = isActive(class_data[i])

print(class_data)

[1 1 1 ... 0 0 0]


In [6]:
df = df[["cid", "class"]]

In [7]:
!pip install pubchempy
import os
import time
import pandas as pd
import pubchempy as pcp
print(os.getcwd())

  Created wheel for pubchempy: filename=PubChemPy-1.0.4-cp37-none-any.whl size=13837 sha256=c2a9c2a7d58f78a35b9d895ad5d70d36f2cfc8758c42cf377381c78cee26a04f
  Stored in directory: /root/.cache/pip/wheels/10/4d/51/6b843681a9a5aef35f0d0fbce243de46f85080036e16118752
Successfully built pubchempy
/content


In [8]:
df_active = df.loc[df['class'] == 1]
df_active

,cid,class
0,31307,1
1,5074,1
2,5754,1
3,2708,1
4,2284,1
...,...,...
5307,1550234,1
5308,1550234,1
5309,15941759,1
5310,15941759,1


In [9]:
df_inactive = df.loc[df['class'] == 0].head(5312)
df_inactive

,cid,class
9458,2123,0
9459,4847,0
9460,4891,0
9461,4495,0
9462,3639,0
...,...,...
14765,663729,0
14766,663731,0
14767,663704,0
14768,663736,0


In [10]:
def to_smiles(data):
    smiles = []
    i = 0
    for c in data.cid:
        out = pcp.Compound.from_cid(c)
        smiles.append(out.isomeric_smiles)
        #print out.cid
        #print out.isomeric_smiles
        i = i+1
        if i % 100 == 0:
          print(i)
        elif i % 10 == 0:
          print("*", end = '')
        else:
          print(".", end = '')
        # time.sleep(0.5)
    return smiles

In [11]:
df_active['smiles'] = to_smiles(df_active)
df_active

.........*.........*.........*.........*.........*.........*.........*.........*.........*.........100
.........*.........*.........*.........*.........*.........*.........*.........*.........*.........200
.........*.........*.........*.........*.........*.........*.........*.........*.........*.........300
.........*.........*.........*.........*.........*.........*.........*.........*.........*.........400
.........*.........*.........*.........*.........*.........*.........*.........*.........*.........500
.........*.........*.........*.........*.........*.........*.........*.........*.........*.........600
.........*.........*.........*.........*.........*.........*.........*.........*.........*.........700
.........*.........*.........*.........*.........*.........*.........*.........*.........*.........800
.........*.........*.........*.........*.........*.........*.........*.........*.........*.........900
.........*.........*.........*.........*.........*.........*.........*...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,cid,class,smiles
0,31307,1,C[C@]12C[C@@H]([C@]3([C@H]([C@@H]1C[C@H]([C@@]...
1,5074,1,CC1=C(C(=O)N2C=CSC2=N1)CCN3CCC(=C(C4=CC=C(C=C4...
2,5754,1,C[C@]12CCC(=O)C=C1CC[C@@H]3[C@@H]2[C@H](C[C@]4...
3,2708,1,C1=CC(=CC=C1CCCC(=O)O)N(CCCl)CCCl
4,2284,1,C1=CC(=CC=C1C(CC(=O)O)CN)Cl
...,...,...,...
5307,1550234,1,COC1=C(C=CC(=C1)/C=C\2/C(=O)/C(=C/C3=CC(=C(C=C...
5308,1550234,1,COC1=C(C=CC(=C1)/C=C\2/C(=O)/C(=C/C3=CC(=C(C=C...
5309,15941759,1,C/C(=C(\C=C\C1=CC(=C(C=C1)O)OC)/O)/C(=O)/C=C/C...
5310,15941759,1,C/C(=C(\C=C\C1=CC(=C(C=C1)O)OC)/O)/C(=O)/C=C/C...


In [12]:
df_inactive['smiles'] = to_smiles(df_inactive)
df_inactive

.........*.........*.........*.........*.........*.........*.........*.........*.........*.........100
.........*.........*.........*.........*.........*.........*.........*.........*.........*.........200
.........*.........*.........*.........*.........*.........*.........*.........*.........*.........300
.........*.........*.........*.........*.........*.........*.........*.........*.........*.........400
.........*.........*.........*.........*.........*.........*.........*.........*.........*.........500
.........*.........*.........*.........*.........*.........*.........*.........*.........*.........600
.........*.........*.........*.........*.........*.........*.........*.........*.........*.........700
.........*.........*.........*.........*.........*.........*.........*.........*.........*.........800
.........*.........*.........*.........*.........*.........*.........*.........*.........*.........900
.........*.........*.........*.........*.........*.........*.........*...

,cid,class,smiles
9458,2123,0,CN(C)C1=NC(=NC(=N1)N(C)C)N(C)C
9459,4847,0,CC1=C(C(=O)N2C=CC=CC2=N1)CCN3CCC(CC3)C(=O)C4=C...
9460,4891,0,C1CCC(CC1)C(=O)N2CC3C4=CC=CC=C4CCN3C(=O)C2
9461,4495,0,CS(=O)(=O)NC1=C(C=C(C=C1)[N+](=O)[O-])OC2=CC=C...
9462,3639,0,C1NC2=CC(=C(C=C2S(=O)(=O)N1)S(=O)(=O)N)Cl
...,...,...,...
14765,663729,0,CCCCNC(=O)C1=CC2=C(N=C3C(=CC=CN3C2=O)C)N(C1=N)...
14766,663731,0,COCCCNC1=C(N=C(O1)C2=CC=CC=C2OC)C#N
14767,663704,0,CCCN1C=NC2=C(C1=O)C3=NC4=CC=CC=C4N=C3N2CCC5=CC...
14768,663736,0,COC(=O)[C@H](CC1=CC=CC=C1)NC(=O)N2CCN(CC2)CC3=...


In [13]:
from google.colab import files
df_active.to_csv('df_active.csv', index=False)
df_inactive.to_csv('df_inactive.csv', index=False)

In [14]:
files.download('df_active.csv')
files.download('df_inactive.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>